### VNLP Turkish Sentiment Analyzer

Introducing VNLP, the cutting-edge sentiment analysis product that helps you understand the sentiment and emotions behind Turkish text. Our state-of-the-art natural language processing technology analyzes text data quickly and accurately, enabling you to gain insights into your customers’ opinions, attitudes, and emotions. With our intuitive user interface, you can easily upload your Turkish text data and get comprehensive analysis results in seconds. VNGRS is designed with simplicity and ease of use in mind, so you can quickly and easily make data-driven decisions to improve your overall customer experience. Trust VNGRS to take your business to the next level!

### Prerequisite


To run this algorithm you need to have access to the following AWS Services:

- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.

### Set up the environment

Here we specify a bucket to use and the role that will be used for working with SageMaker.



In [3]:
# S3 prefix
prefix = "vnlp-model-serving-marketplace"

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [4]:
from sagemaker.model import Model
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.predictor import Predictor
import json
from sagemaker import ModelPackage


### Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.



In [5]:
import sagemaker as sage
from time import gmtime, strftime

In [6]:
sagemaker_session = sage.Session()
#bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")

### Create a deployable model from model package

In [8]:
modelpackage_arn = 'arn:aws:sagemaker:eu-west-1:707858255059:model-package/vnlp-sentiment-validated-1679584123'

In [9]:
sentiment_analyses_model = ModelPackage(
    role=role,
    model_package_arn=modelpackage_arn,
    sagemaker_session=sagemaker_session,
)

### Deploy an endpoint with the SageMaker model that is created

In [10]:
endpoint_name = "vnlp-sentiment-analyses-endpoint"

In [11]:
predictor_sentiment_analyser = sentiment_analyses_model.deploy(
    1, "ml.c4.2xlarge", endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

----!

### Perform real-time inference on the model.

In [16]:
payload = '{"sentiment": "Gerçek tarafsız kaliteli soft power gazetecilik iyiki varsın"}'


In [17]:
response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType="application/json",
                                   Body=payload)

In [18]:
sent_analyse_result = response["Body"].read().decode()

In [19]:
print(sent_analyse_result)

{"prediction": 0.9991540908813477}
